In [2]:
import pandas as pd

In [4]:
january_trips=pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [5]:
january_trips.shape

(3066766, 19)

In [6]:
january_trips.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [ ]:
import pandas as pd

january_trips['tpep_pickup_datetime'] = pd.to_datetime(january_trips['tpep_pickup_datetime'])
january_trips['tpep_dropoff_datetime'] = pd.to_datetime(january_trips['tpep_dropoff_datetime'])

january_trips['duration'] = (january_trips['tpep_dropoff_datetime'] - january_trips['tpep_pickup_datetime']).dt.total_seconds() / 60

std_duration = january_trips['duration'].std()

std_duration


np.float64(42.59435124195458)

In [17]:
filter_data=january_trips[(january_trips['duration'] <=60) & (january_trips['duration'] >=1)]

proportion_outliers=filter_data.shape[0]/len(january_trips)*100
proportion_outliers


98.1220282212598

In [18]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']
filter_data[categorical] = filter_data[categorical].astype(str)
data_dicts = filter_data[categorical].to_dict(orient='records')
vectorizer = DictVectorizer()
feature_matrix = vectorizer.fit_transform(data_dicts)
print("Shape of the Feature Matrix:", feature_matrix.shape)


C:\Users\57300\AppData\Local\Temp\ipykernel_5676\1716243837.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data[categorical] = filter_data[categorical].astype(str)


Shape of the Feature Matrix: (3009173, 515)


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


y = filter_data['duration']

model = LinearRegression()
model.fit(feature_matrix, y)
predictions = model.predict(feature_matrix)
rmse = np.sqrt(mean_squared_error(y, predictions))
print(f"RMSE: {rmse}")


RMSE: 7.6492619533128225


In [22]:
february_trips=pd.read_parquet('data/yellow_tripdata_2023-02.parquet')


In [23]:
february_trips['duration'] = (february_trips['tpep_dropoff_datetime'] - february_trips['tpep_pickup_datetime']).dt.total_seconds() / 60
february_trips = february_trips[(february_trips['duration'] <= 60) & (february_trips['duration'] >= 1)]



In [24]:
february_trips[categorical] = february_trips[categorical].astype(str)
data_dicts_feb = february_trips[categorical].to_dict(orient='records')
feature_matrix_feb = vectorizer.transform(data_dicts_feb)

In [25]:
february_predictions = model.predict(feature_matrix_feb)
rmse_feb = np.sqrt(mean_squared_error(february_trips['duration'], february_predictions))

In [26]:
rmse_feb

np.float64(7.811816520976144)